# 03_funcpp02

功能像预处理的最核心部分。这一部分我们完成一下步骤
* Despike
* Slice-timing correction (时间层矫正)
* Distortion correction (图像畸变矫正)
* Motion correction (头动矫正)
* align anat 2 epi (结构像到功能像的配准) 
* 把所有epi划到个体结构像空间
* 把所有epi的volume数据划到surface data
* 把所有epi的划到MNI标准空间
* Scale (标准化数据尺度，抓换成percent signal change)

## Step 1. Despike

In [ ]:
# ================================ despike =================================
# 产生新的P01文件  带有despike标记
# 这里应该利用了上一步生成的最小outlier volume信息， 同时基于pb00进行运算，生成pb01
# 此时的pb01后缀为  r0*+despike+orig
# 这一步仅生成pb01，不对前面的数据做修改
# 然后删除pb00
for run in runstr:
    ! 3dDespike -NEW -nomask -prefix pb01.{subj}.r{run}.despike pb00.{subj}.r{run}.tcat+orig
#! rm pb00*  #不建议删除pb00 因为总会出现奇怪的问题， 保留pb00可以从这里重新开始


# pb00是未做despike的数据，仅仅做了平移，和T1像做了空间上的对齐， 而T1像已经和MNI做了对齐
# pb00是一个立体的图像，包含了大脑，也包含大脑之外的组织

## Step 2. Slice-timing correction

In [ ]:
# ================================= tshift =================================
# 因为每个tr里，扫描N层，  每一层扫描的时间不一样，并且可能扫描顺序是隔层或者逐层
# 这里需要把每一层的扫描时间校正成同一个时间（可以指定以所有TR的某一层为准，或者按默认来）
# 跑完后会变成pb02  后缀变为  tshift+orig  表示时间time发生了平移shift
# 删除pb01
for run in runstr:
    ! 3dTshift -tzero 0 -quintic -prefix pb02.{subj}.r{run}.tshift \
         -verbose -tpattern @{'../SliceTiming.txt'} pb01.{subj}.r{run}.despike+orig

# 删掉上一步despike的data节省空间
! rm pb01*

## Step 3. Distortion correction

因为磁场不均匀的关系，图像会出现畸变。现在来做矫正。

In [ ]:
# 这里还没有使用场图
# 使用正扫和反扫两两成对来互相校正
# 这一步有3种方法， 1是使用场图， 2是AP-PA互校正 3是afni的方法纯粹从图像来进行校正（另一种AP-PA的用法？）
# 这里生成blip_pre_*_mean, blip_post_*_mean， 可以用来进行校正效果的验证
# 这里生成的blip_warp文件是主要所得，应该是对所有数据的校正结果
# 如果关注前额叶， 则应该选用PA，这样的话前额叶部分会伸出去，然后再校正里压回来，这样信息不会丢失
# 使用pb02， 生成blip_warp和blip_pre blip_post打头的文件

for fwd,rev in zip(fwdfiles, revfiles):
        # create median datasets from forward and reverse time series
        ! 3dTstat -median -prefix rm.blip.med.fwd.r{fwd} pb02.{subj}.r{fwd}.tshift+orig
        ! 3dTstat -median -prefix rm.blip.med.rev.r{rev} pb02.{subj}.r{rev}.tshift+orig
        #对一个AP方向扫描的run的所有TR，取一个中位数
        #对一个PA方向扫描的run的所有TR，取一个中位数

        # automask the median datasets 
        ! 3dAutomask -apply_prefix rm.blip.med.masked.fwd.r{fwd} rm.blip.med.fwd.r{fwd}+orig
        ! 3dAutomask -apply_prefix rm.blip.med.masked.rev.r{rev} rm.blip.med.rev.r{rev}+orig
        # 使用3dAutomask命令， 对前面获取的中位数结果剥头皮，并且使用apply_prefix命令应用剥头皮结果，得到的是剥头皮后的结果（其值是有意义的激活值而不是0或1）

        # compute the midpoint warp between the median datasets
        ! 3dQwarp -plusminus -pmNAMES Rev.r{rev} Fwd.r{fwd}                           \
                -pblur 0.05 0.05 -blur -1 -1                          \
                -noweight -minpatch 9                                 \
                -noXdis -noZdis                                       \
                -source rm.blip.med.masked.rev.r{rev}+orig                   \
                -base   rm.blip.med.masked.fwd.r{fwd}+orig                   \
                -prefix blip_warp
        
        ! 3dNwarpApply -quintic -nwarp blip_warp_Fwd.r{fwd}_WARP+orig        \
                -source rm.blip.med.masked.fwd.r{fwd}+orig               \
                -prefix rm.blip.med.masked.fwd.post.r{fwd}

        ! 3dNwarpApply -quintic -nwarp blip_warp_Rev.r{rev}_WARP+orig        \
                -source rm.blip.med.masked.rev.r{rev}+orig               \
                -prefix rm.blip.med.masked.rev.post.r{rev}
        
        # 删掉多余文件
        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* 

        # 修改个名字以便以后的操作
        ! 3dcopy blip_warp_Fwd.r{fwd}_WARP+orig blip_warp.r{fwd}_WARP+orig
        ! 3dcopy blip_warp_Rev.r{rev}_WARP+orig blip_warp.r{rev}_WARP+orig

        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* blip_warp_Fwd.r{fwd}_WARP+orig* blip_warp_Rev.r{rev}_WARP+orig* 

#此处生成的blip_warp.r* 是distortion correction的转换矩阵，每一个run都对应了一个矩阵  
# 在后面将被用于把所有的run的EPI从pb02的状态转换到中间值（也就是矫正了AP扫描和PA扫描带来的形变问题）

# for QC check，我们生成一个校正前和矫正后的平均EPI数据来作为distortion correction的效果证明
! 3dMean -prefix blip_pre_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.r*
! 3dMean -prefix blip_pre_rev.epi_mean.nii.gz rm.blip.med.masked.rev.r*
! 3dMean -prefix blip_post_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.post.r*
! 3dMean -prefix blip_post_rev.epi_mean.nii.gz rm.blip.med.masked.rev.post.r*

#此处的blip_post_fwd是所有fwd方向的epi的平均值，并且已经被剥了头皮，（但是剥的比较粗糙）

# remove redundent file
! rm rm*

这一步完成之后，可以手动打开
* blip_pre_fwd.epi_mean.nii.gz
* blip_pre_rev.epi_mean.nii.gz
* blip_post_fwd.epi_mean.nii.gz
* blip_post_rev.epi_mean.nii.gz

这四个文件也记录了对侧矫正的效果，只要有矫正效果，没有什么大问题即可。

然后把所有的run都做correction

In [ ]:
# 在这里将配准用的计算结果应用到图像数据里
# 结合前面生成的矫正数据 与 pb02数据
# 生成新的pb03.blip
for run in runstr:
    ! 3dNwarpApply -quintic -nwarp blip_warp.r{run}_WARP+orig      \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix pb03.{subj}.r{run}.blip

#使用上一步生成的blip_warp转换矩阵把pb02的EPI数据转换为pb03的EPI数据

## Step 4. Motion correction 

首先做motion correction。但是我们并不用其结果，主要是得到mc的线性转换矩阵

In [ ]:
#生成几个文件：
# mat.r0*.vr.aff12.1D
# dfile.r0*.1D
# rm.epi.
# 可能对pb03做了修改， 因为有overwrite一步

# ================================= volreg =================================
base = 'vr_base_min_outlier+orig' #在前面找到的  也就是那个最小头动的tr

# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # register each volume to the base image
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}.1D \
             -cubic \
             -1Dmatrix_save mat.r{run}.vr.aff12.1D \  
             -prefix rm.epi.nomask.r{run} pb03.{subj}.r{run}.blip+orig
    # 这一步也生成了 rm.epi.nomask.r{run}， 这个数据是对pb03进行了motion correction操作后得到的， 同时这一步操作使用的motion correction转换矩阵也被保存了
    # 此处的 rm.epi.nomask.r{run}数据是先做了distortion correct后的数据(即pb03)，再经过此处的motion correction所得的结果
    # 这个数据在后续处理中用于生成pb04

    # 在这里也保存了mat.r{run}.vr.aff12.1D作为motion correction操作的转换矩阵，
    # 后续会和前面的distortion correction转换矩阵一起进行EPI到标准空间的转换， 但不会和distortion correction转换矩阵合并（因为这个矩阵是线性变换，而distortion correction转换矩阵非线性
    

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb03.{subj}.r{run}.blip+orig -expr 1 -prefix rm.epi.all1
    # 在这里获取pb03的mask，其名称为 rm.epi.all1， 方法是直接获取pb03里并把油数据的地方标记为1，没有数值的地方标记为0
    # 注意，此处获取的pb03的all1数据并不是剥头皮的后的mask，因为pb03本身也不是一个被剥头皮的数据
    # 此处的pb03是做了distortion correct后的数据，而在这里取的all1数据，只是EPI数据被裁掉了一些边角后的结果，不可以和剥头皮后的mask混淆
    # 

    # warp the all-1 dataset for extents masking
    ! 3dAllineate -base {base} \
                -input rm.epi.all1+orig \
                -1Dmatrix_apply mat.r{run}.vr.aff12.1D \
                -final NN -quiet \
                -prefix rm.epi.1.r{run}
    #此处把rm.epi.all1作为输入， 应用motion correctio转换矩阵，把all1矩阵也做一次转换，然后得到rm.epi.1.r{run}
    #注意这里的rm.epi.1.r{run}应该包含每一个run的信息，并且每个run下都是4D数据，其中包含每一个TR的all1结果

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig
    # 对rm.epi.1.r{run}做计算， 获取每一个run里所有TR的all1矩阵的交集，  以此确保这个run里的所有TR数据都是有效数据，不包含任何all1以外的数据
    # 此处得到的rm.epi.min.r{run} 数据应该包含8个run，且每个run下都是3D数据，为一个0 1构成的all1矩阵
    # 仍然要强调一下，此处的all1矩阵不是剥头皮的mask，而是一个去掉EPI边角后的结果， 而去掉EPI边角的原因是EPI数据被做过一次distortion correction



计算更多的头动数值，以后可以用来GLM中的regressor

In [ ]:
# --------------- deal with motion parameter -------------------
import numpy as np
# 生成pb04.subj.r0*+volreg+orig
# 生成epi_mean.nii.gz
# motion打头的几个文件
# 生成mc文件夹，里面放了矫正结果比较
# we take dmeaned motion (6), demean motion derivative (6) and their squares (12)
# 合并 motion params

! cat dfile.r*.1D > dfile_rall.1D
# 计算去均值的motion parameters (for use in regression)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -demean -write motion_demean.1D
# 计算一阶导数 (just to have)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -derivative -demean -write motion_deriv.1D
# calculate the square of demean and derivative of motion parameters (just to have)
# for resting preproc, we usually have 24 motion regressor (6motion+6deriv+12 their square)
np.savetxt('motion_demeansq.1D', np.loadtxt('motion_demean.1D')**2)
np.savetxt('motion_derivsq.1D', np.loadtxt('motion_deriv.1D')**2)
# create censor file motion_${subj}_censor.1D, for censoring motion
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} \
   -show_censor_count -censor_prev_TR \
   -censor_motion {motion_censor} motion_{subj}


# Estimate the motion parameter after motion correction, we can check the
# effects of MC
! mkdir mc
for run in runstr:
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}_pos.1D \
             rm.epi.nomask.r{run}+orig
    ! rm volreg+orig*
#此处用rm.epi.nomask.r{run}+orig数据生成一些检查信息，用来判断预处理的效果, 意义不大

# make a single file of motion params
! cat dfile.r*_pos.1D > dfile_rall_pos.1D


import matplotlib.pyplot as plt
# make figure pre mc
dfile_pre = np.loadtxt('dfile_rall.1D')
plot(range(dfile_pre.shape[0]), dfile_pre[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pre.pdf');plt.close('all')

plot(range(dfile_pre.shape[0]), dfile_pre[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pre.pdf');plt.close('all')
# make figure pos mc
dfile_pos = np.loadtxt('dfile_rall_pos.1D')
plot(range(dfile_pos.shape[0]), dfile_pos[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pos.pdf');plt.close('all')
plot(range(dfile_pos.shape[0]), dfile_pos[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pos.pdf');plt.close('all')

# move file to directory
#将绘图结果和头动校正数据保存到mc里
! mv rots*.pdf tran*.pdf dfile.r*_pos.1D dfile_r*_pos.1D mc/

# note TRs that were not censored, note ktrs here is a str
ktrs = unix_wrapper(f'1d_tool.py -infile motion_{subj}_censor.1D \
                       -show_trs_uncensored encoded', wantreturn=True, verbose=0)

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig and apply the task
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# 对8个run的EPI的最小mask取一个均值，得到rm.epi.mean


! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents
# 猜测： 可能由于前一步生成rm.epi.mean数据时候是把8个run做了平均，因此存在部分区域的值是0-1之间的小数，而这些区域应当舍弃并变为0
# 因此把上述rm.epi.mean数据全部减去0.999，然后把剩余大于0的数据全部变为1，而小于0的数据变为0
# 这里生成的mask_epi_extents实际上是8个run的all1矩阵的交集，此处叫做mask_epi_extents有一定误导性，要注意这里的mask和剥头皮后的mask无关

# and apply the extents mask to the EPI data
# (delete any time series with missing data)
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb04.{subj}.r{run}.volreg
! rm -f rm.*  # rm.epi.nomask are big files, remove them
! rm -f {base}* 
# 这里对于前文生成的nomask数据————rm.epi.nomask.r{run}进行处理，得到pb04，  
# 具体过程是把nomask数据和前面得到的mask_epi_extents相乘，
# 目前得到的pb04已经过以下处理：  distortion correct、 motion correct、 对所有TR做裁剪， 使得8个run里的每一个TR里包含的脑子的尺寸和位置都可以重叠

# calculate a mean epi volume for next step anat epi registration
# 这是旧的方法，其过程是先对多个run进行run之间的平均，  然后再对平均run的结果进行时间维的平均， 但这么做存在问题，即要求所有run的长度必须一致，否则无法进行run间平均
#! 3dMean -prefix rm.epi_mean.nii.gz pb04.{subj}.r*.volreg+orig.HEAD
#! 3dTstat -prefix epi_mean.nii.gz rm.epi_mean.nii.gz

# 以下是新方法， 先对所有run的结果分别做时间维的平均，然后再对平均后的多个结果进行run间平均
for run in runstr:
    ! 3dTstat -prefix rm.epi_mean_each_run.{subj}.r{run} pb04.{subj}.r{run}.volreg+orig.HEAD
! 3dMean -prefix epi_mean.nii.gz rm.epi_mean_each_run.{subj}.r*
# 把每一个run下的的所有TR的结果（pb04）进行时间维度的平均
# 然后把所有run的平均结果再做平均， 由此得到一个3D数据，即所有EPI的平均值， 注意这里的平均结果仍然是没有做过剥头皮的


! rm rm* mask_epi_extents+orig*

# 删掉上一步distortion correction的数据，节省空间
! rm pb03*

## Step 5. anat2epi

这一步非常重要，是前面处理好的功能像和结构像进行配准，配准的结果一定要手动检查

In [ ]:
# 首先把用来配准的epi像make a copy
! 3dcopy epi_mean.nii.gz epi_mean_tmp.nii.gz


In [ ]:
# 关键步骤。我们是把结构像配准到功能像，但是我们得到的转换矩阵是从相反的，功能像到结构像，这个矩阵可以后面用来转化epi数据
# 生成了{subj}_SurfVol_al打头的文件
# 生成了{subj}_SurfVol_ns打头的文件
# 如果要重复进行这一步，需要删掉这两类文件


! align_epi_anat.py -anat2epi -anat {t1.str} \
-save_skullstrip -suffix _al_junk \
-epi epi_mean_tmp.nii.gz -epi_base 0 \
-epi_strip 3dAutomask \
-volreg off -tshift off -giant_move
# 此处输入的T1像（{t1.str}，以及平均epi像（epi_mean_tmp.nii.gz） 都是包含了头皮的数据
# 此处的epi_strip命令，指定了3dAutomask方法对平均的EPI数据做去头皮，
# 同时T1像也被默认做了剥头皮操作

# 此处会得到{subj}_SurfVol_al_junk_mat.aff12.1D,  为结构像到功能像的转换矩阵， （ 注意代码中的命令“anat2epi”）
# 得到 {subj}_SurfVol_al_junk.nii.gz   为T1像被配准到EPI的结果， 
# 得到 {subj}_SurfVol_ns.nii.gz   为T1像被剥头皮的结果


# 生成nifti文件，方便我们在fsleyes上面查看配准效果
! 3dcopy {subj}_SurfVol_al_junk+orig {subj}_SurfVol_al_junk.nii.gz
# 此处把T1配准到EPI的结果转换为nii格式，然后在flseyes里打开，需要与epi_mean_tmp数据在进行对比检查两者是否一致，如果不匹配，则需要重新进行一次配准


完成上面一步之后，要打开afni仔细检查配准的结果，其中underlay选择CN003_SurfVol_al_junk.nii, overlay选择epi_mean_tmp+orig。然后检查两者是否一致。如果不匹配，则需要重新进行上面的过程，在开始之前，需要先删除上面产生的文件。可以做
! rm {subj}_SurfVol_al* {subj}_SurfVol_ns*
确认配准没有问题之后，进行下一步操作

In [ ]:
# note that we align anat 2 epi, however, this xfm is from epi space to anat space
# note that this xfm is compatible with LPS+ space, which is the default space in AFNI
# 删除上一步生成的几个文件， 只保留 subj_SurfVol_al_junk.nii.gz
# 生成anat_final   anat2epi  mat.epi2anat  epi2anat  volumized..等以及4张图片
# 不清楚 mat.anat2epi.aff12.1D是什么时候生成的，看时间应该是上一步生成的，但看名称上一步里并没有它
# 此处的t1和funcpp01的step1里的t1不同，  实际上的t1在step2里被更新了 后缀多了+orig

! mv {t1.strnosuffix[:-5]}_al_junk_mat.aff12.1D mat.anat2epi.aff12.1D
# 把前面生成的T1-EPI的转换矩阵重命名为mat.anat2epi.aff12.1D


# create an skull-striped anat_final dataset, aligned with stats
! 3dcopy {t1.strnosuffix[:-5]}_ns+orig anat_final.{subj}.nii.gz
! rm {t1.strnosuffix[:-5]}_ns+orig*
#把剥头皮后的T1像复制为anat_final.{subj}.nii.gz


# rewrite the name of aligned anat
! 3dcopy {t1.strnosuffix[:-5]}_al_junk+orig anat2epi.{subj}.nii.gz
! rm {t1.strnosuffix[:-5]}_al_junk+orig*
#把T1-EPI的配准结果重命名为  anat2epi.{subj}.nii.gz


# Invert xfm
! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I > mat.epi2anat.aff12.1D
# 把T1-EPI的转换矩阵反转， 得到EPI-T1的转换矩阵 mat.epi2anat.aff12.1D


# 前面是将结构像绘制到EPI，现在是从EPI绘制到结构像
# warp the volreg base EPI dataset back to anat to make a final version
! 3dAllineate -base anat_final.{subj}.nii.gz \
            -input epi_mean.nii.gz \
            -1Dmatrix_apply mat.epi2anat.aff12.1D \
            -prefix epi2anat.{subj}.nii.gz
# 基于被剥头皮后的T1像(似乎不需要这个base？ 不理解为什么，也许是用来做参考)
# 输入的是平均epi结果：epi_mean.nii.gz
# 基于epi2anat转换矩阵
# 得到的是epi2anat.{subj}.nii.gz， 也就是平均epi被配准到T1像的结果
# epi2anat.{subj}.nii.gz在下面被用于生成检查图片，方便查看转换结果是否正常


# Record final registration costs
! 3dAllineate -base epi2anat.{subj}.nii.gz -allcostX -input anat_final.{subj}.nii.gz > out.allcostX.txt
# 此处意义不明，可能用于后期对数据预处理的质量做诊断

# Take the snapshots to show the quality of alignment
! @snapshot_volreg epi2anat.{subj}.nii.gz anat_final.{subj}.nii.gz
! @snapshot_volreg anat_final.{subj}.nii.gz epi2anat.{subj}.nii.gz
# 此处把 平均epi被配准到T1像的结果 epi2anat    与 T1的剥头皮结果anat_final  互相做了对比（互为上下）

! @snapshot_volreg anat2epi.{subj}.nii.gz epi_mean.nii.gz
! @snapshot_volreg epi_mean.nii.gz anat2epi.{subj}.nii.gz
# 此处把T1配准到EPI的结果， 与 EPI均值 互相做了对比

## Step 6. epi to individual anat

现在我们有了epi到anat的配准，然后我们就可以把所有的epi的文件画到t1的空间。注意，这一步并不是从motion correction之后的文件，也就是pb03.CN003.r0X.volreg+orig。我们要从早distrotion correction之前的文件，把从DC-MC-epi2anat这三个转换一次性的做完

In [ ]:
# 生成几个准备删除的文件，以及mat.r0*.warp.aff12.1D文件
# 
resolution=2.4  #这个值指定了EPI被配准到T1后的体素尺寸，单位mm

# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # concatenate MC+aff transforms
    ! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I mat.r{run}.vr.aff12.1D > mat.r{run}.warp.aff12.1D
    # 首先，合并motion correction的转换矩阵：mat.r{run}.vr.aff12.1D， (注意这里每个run都有一个单独的MC转换矩阵, 再注意这里的命令 -I表示对MC转转矩阵做反转)
    # 以及 T1到EPI的转换矩阵：mat.anat2epi.aff12.1D 
    # 因为这两个转换矩阵是线性的，因此可以直接合并， 最终生成 mat.r{run}.warp.aff12.1D,  每个run单独得到一个转换矩阵
    # 这里的结果是  MC和anat2epi两个转换矩阵的总和（线性变换）
    #尚不清楚为什么这里会指定 -I

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1} -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)
    
    # 此处的命令：3dNwarpApply可执行非线性转换
    # 这里把上一步的线性变换矩阵，以及之前生成的非线性变换矩阵blip_warp (也就是distortion correction矩阵)，一并输入unix_wrapper(cmd)进行变换
    # 这里被变换的文件是pb02，也就是仅仅做了despike处理和Slice-timing correction处理的文件
    # 最后会得到一个run内的EPI被配准到T1的结果
    # 这里的mast指定的是该名被试的t1数据，yyq说这里要根据t1数据获取grid信息，最后生成的数据会和t1保持相同的grid


    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1
    # 然后对pb02获取一个all1矩阵，  道理同STEP4 motion correction

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1} -dxyz {resolution} \
            -ainterp NN -quiet \
            -source rm.epi.all1+orig \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)
    # 对all1矩阵也同样的做一次配准,

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig
        # 对转换后的all1矩阵取每个run下的交集 道理同STEP4 motion correction

    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+orig*


# 这部分代码的操作，实际上是把每一个被试的EPI数据，配准到了这个被试自己的T1像上
# 如果要做组水平分析，这一步似乎用处不大
# 组水平分析需要把每一个被试的EPI都配准到标准MNI空间里


做了变化之后，也要处理一下mask的问题

In [ ]:
# ----------------------------------------
# 生成pb05  mask_epi_extents+orig
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents
#d 道理同STEP4 motion correction

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb05.{subj}.r{run}.al2anat
    # save to nifti format seems to reduce file size
# 道理同STEP4 motion correction，  注意这里的mask不是剥头皮后的mask
# 确保每一个RUN下的所有TR的脑子的位置和大小都是一致的

# rm.epi.nomask are big files, remove them
! rm rm.*

## Step 7. epi volume 2 surface

上一步我们已经把epi数据划到了和个体的结构像一个空间。很多时候我们需要做基于surface的数据分析。我们进一步把三维的epi数据插值划到surface上

In [ ]:
# 生成 pb05.subj.rh.r0*.surf.niml.dset
# 生成 pb05.subj.lh.r0*.surf.niml.dset

FREESURFER_HOME = os.getenv('FREESURFER_HOME') # 获得 FreeSurfer的主文件夹路径
surface_dir = FREESURFER_HOME+f'/subjects/{subj}/SUMA'
# map volume data to the surface of each hemisphere
for hemi in ['lh', 'rh']:
    for  run in runstr:
        ! 3dVol2Surf -spec {surface_dir}/std.141.{subj}_{hemi}.spec   \
                   -sv {subj}_SurfVol+orig           \
                   -surf_A smoothwm                            \
                   -surf_B pial                                \
                   -f_index nodes                              \
                   -f_steps 10                                 \
                   -map_func ave                               \
                   -oob_value 0                                \
                   -grid_parent pb05.{subj}.r{run}.al2anat+orig   \
                   -out_niml pb05.{subj}.{hemi}.r{run}.surf.niml.dset

## Step 8. epi to MNI anat

In [ ]:
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
# 生成许多待删除的文件，  rm.epi...

for run in runstr:

    # we apply distortion correction + motion correction + anat2epi + nonlinear warp to MNI transformation
    # 注意我们这里是相反的顺序来concatenate的
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)
        # 此处需要输入：
        # anatQQ.{subj}_WARP.nii  T1-MNI 的非线性转换矩阵（需确认）
        # anatQQ.{subj}.aff12.1D  T1-MNI 的线性转换矩阵（需确认）
        # mat.r{run}.warp.aff12.1D   T1-EPI的线性转换矩阵 结合  EPI的motion correction线性转换矩阵
        # blip_warp.r{run}_WARP+orig  EPI做distortion correction的非线性转换矩阵
        # 疑问，这里的几个转换矩阵的方向很怪，  好像不能直接用来做转换， 是否要对其中的几个矩阵也做一次反转处理？ 应该需要检查一下unix_wrapper函数的内容

        # 这里被转换的数据是pb02，也就是仅仅做了despike处理和Slice-timing correction处理的文件
        # master指定的是MNI标准空间模板（不清楚其作用是什么，pb02直接经过转换后就可以得到最终结果了吧？为什么还要指定MNI？）yyq说这里要根据MNI数据获取grid信息，最后生成的数据会和MNI保持相同的grid

        #这里得到的nomask数据是EPI数据被转换到MNI标准空间里的数据， 注意这里的nomask指的是没有被裁剪过，而不是没有被剥头皮（确实没有剥头皮）


    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1
        #此处获取all1矩阵，道理同上

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source rm.epi.all1+orig         \
            -ainterp NN -quiet \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)
    #此处对all1矩阵做同样的变换，道理同上

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+tlrc
        #此处对all1矩阵，取每个run下的所有TR里的交集

    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+tlrc*

做了变化之后，也要处理一下mask的问题

In [ ]:
# ----------------------------------------
# 生成pb06.subj.r0*.al2mni+tlrc
# 以及mask_epi_2mni_extents+tlrc
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+tlrc -expr "step(a-0.999)" -prefix mask_epi_2mni_extents
#使用熟悉的操作，获取8个run下的所有TR的交集

# 去掉mask之外的voxel的数据， 注意这里的mask仍然和剥头皮无关
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+tlrc -b mask_epi_2mni_extents+tlrc \
           -expr "a*b" -prefix pb06.{subj}.r{run}.al2mni
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*

In [ ]:
# 补充步骤，用于进行smooth操作，  2022.7.3添加
# 目的是对上一步step8所生成的pb06进行smooth   生成pb06s
# 再对上上一步step7所生成的pb05***surf.niml文件进行smooth  生成pb05s*rh  和 pb05s*lh
# 再对上上一步step7所生成的pb05.subj.r0*.al2anat+orig文件进行smooth  生成pb05s*.subj.r0*.blur+al2anat+orig
# pb05是配准到个体的T1的EPI
# pb06是配准到标准MNI的EPI

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb06s.{subj}.r{run}.blur+al2mni+tlrc \
        pb06.{subj}.r{run}.al2mni+tlrc

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.lh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.lh.r{run}.surf.niml.dset

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.rh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.rh.r{run}.surf.niml.dset

#以下代码可以不用跑
for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.r{run}.blur+al2anat+orig \
        pb05.{subj}.r{run}.al2anat+orig


'''  
#afni官网上的对于surface数据进行smooth的代码

foreach hemi ( lh rh )
    foreach run ( $runs )
        # to save time, estimate blur parameters only once
        if ( ! -f surf.smooth.params.1D ) then
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -target_fwhm 6.0                                  \
                       -blurmaster pb03.$subj.$hemi.r$run.surf.niml.dset \
                       -detrend_master                                   \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset     \
                       | tee surf.smooth.params.1D
        else
            set params = `1dcat surf.smooth.params.1D`
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -Niter $params[1]                                 \
                       -sigma $params[2]                                 \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset
        endif
    end
end
'''

## Step 9. 创造一个所有epi和mni模板共有的mask

In [ ]:
# ====================== Mask =================================
# 这里创造mask所用的信息是pb06， 而不是前一步的pb06s

# Create 'full_mask' dataset (union mask)
for run in runstr:
    ! 3dAutomask -prefix rm.mask_r{run}.nii.gz pb06.{subj}.r{run}.al2mni+tlrc
# 这里对pb06的数据，逐个run的剥头皮
# 这里应该是对一个run剥一次头皮就好了吧， 也可能会对每个TR单独剥一次头皮？
# 这里生成的rm.mask_r{run}.nii.gz 是剥头皮后的mask数据（值为0或1），注意和apply_prefix命令不同


# 对所有run下的mask文件取并集，得到full_mask
! 3dmask_tool -inputs rm.mask_r*.nii.gz -union -prefix full_mask.{subj}.nii.gz
! rm rm.mask*

# ---- create subject anatomy mask, mask_anat.$subj+orig ----
#      (resampled from aligned anat)

# 把MNI图像resample到full_mask上
! 3dresample -master full_mask.{subj}.nii.gz -input \
           MNI152_2009_template_SSW.nii.gz -prefix rm.resam.anat.nii.gz


# convert to binary anat mask; fill gaps and holes
! 3dmask_tool -dilate_input 5 -5 -fill_holes -input rm.resam.anat.nii.gz \
            -prefix mask_anat.{subj}.nii.gz
#对resample之后的MNI数据取一次mask，得到mask_anat.{subj}.nii.gz， 即结构像的mask
# fill_holes命令表示把可能存在的空隙填充为1
# dilate_input命令表示把生成的mask做一个扩充，使得其能够包裹更多区间（甚至包裹到一部分大脑之外的组织），提高冗余
# ！！！！！！！！！！这里需要确认，因为得到的mask_anat.{subj}.nii.gz数据，看起来像是对整个头颅的mask，而不是对大脑的mask
# 按理说前一步得到的rm.resam.anat.nii.gz结果应该已经是接近fullmask的样子才对



# 结合功能像和结构像的mask
# compute tighter EPI mask by intersecting with anat mask
! 3dmask_tool -input full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz \
            -inter -prefix mask_epi_anat.{subj}.nii.gz
# 对基于EPI获取的full_mask以及 对MNI做了resample后的mask数据，取一个交集，得到最终mask数据：mask_epi_anat.{subj}.nii.gz 

# note Dice coefficient of masks, as well
! 3ddot -dodice full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz > out.mask_ae_dice.txt
#意义不明

# 删掉多余文件
! rm rm*

## Step 10. scale

原始EPI图像的数据可能很大，从几百到4000不等。但是我们一般不考虑这个绝对强度，考虑的是信号增长的比例。所以把数据scale一下, 让所有的voxel的时间序列均值为100

In [ ]:
# 注意！！！这个cell的代码是对pb06以及pb05 进行scale重置 也就是未经smooth的数据
# scale each voxel time series to have a mean of 100
# (be sure no negatives creep in)
# (subject to a range of [0,200])
# scale volume data

for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06.{subj}.r{run}.al2mni+tlrc
    ! 3dcalc -a pb06.{subj}.r{run}.al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs.{subj}.nii.gz pb07.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05.{subj}.{hemi}.r{run}.surf.niml.dset
       ! 3dcalc -a pb05.{subj}.{hemi}.r{run}.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07.{subj}.{hemi}.r{run}.scale.niml.dset

# remove redundant file
! rm rm.*mean* 

In [ ]:
# 注意！！！这个cell的代码是对pb06s 以及pb05s 进行scale重置 也就是经过了smooth的数据
# 生成pb07s*.scale  
# 这个文件被整合成all_runs_with_smooth
# 生成pb07s*.scale.niml.dset

# scale volume data
for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06s.{subj}.r{run}.blur+al2mni+tlrc
    ! 3dcalc -a pb06s.{subj}.r{run}.blur+al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07s.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs_with_smooth.{subj}.nii.gz pb07s.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset
       ! 3dcalc -a pb05s.{subj}.{hemi}.r{run}.blur.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07s.{subj}.{hemi}.r{run}.scale.niml.dset
# remove redundant file
! rm rm.*mean* 

到这一步，基本上最主要的预处理就完结了。下一步可以进一步针对surface数据跑GLM，因为afni的surface都是在freesurfer的模板上对其且标准化的，所以surface数据可以直接跑group analysis。也可以在已经划到MNI space上的volume数据跑GLM。以后在volume上跑GLM的结果可以直接用MNI152的T1像来可视化